In [1]:
import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import multiprocessing
import itertools as it
from scipy.special import expit
import pymc3 as pm
import arviz as az
import theano.tensor as tt
import warnings
warnings.simplefilter('ignore')

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set()  # Nice plot aesthetic

In [19]:
with pm.Model() as model:

    # Lengthscale
    ρ = pm.HalfCauchy('ρ', 5)
    η = pm.HalfCauchy('η', 5)
    
    M = pm.gp.mean.Linear(coeffs=0)
    K = η**2 * pm.gp.cov.ExpQuad(1, ρ) 
    
    σ = pm.HalfNormal('σ', 50)
    
    gp = pm.gp.Marginal(mean_func=M, cov_func=K)
    gp.marginal_likelihood('f', X=np.linspace(0, 1, 100).reshape(-1, 1), noise=σ, y=np.linspace(0, 1, 100).reshape(-1, 1))
    # gp.marginal_likelihood('y', X=np.linspace(0, 1, 10).reshape(-1, 1), y=u_sorted, noise=σ)

In [20]:
gp.predict(Xnew=np.linspace(0, 1, 100).reshape(-1, 1), point={'ρ': 1, 'η': 1})

[array([[0.02667497, 0.02667497, 0.02667497, ..., 0.02667497, 0.02667497,
         0.02667497],
        [0.02684879, 0.02684879, 0.02684879, ..., 0.02684879, 0.02684879,
         0.02684879],
        [0.02702114, 0.02702114, 0.02702114, ..., 0.02702114, 0.02702114,
         0.02702114],
        ...,
        [0.03193841, 0.03193841, 0.03193841, ..., 0.03193841, 0.03193841,
         0.03193841],
        [0.03184628, 0.03184628, 0.03184628, ..., 0.03184628, 0.03184628,
         0.03184628],
        [0.03175134, 0.03175134, 0.03175134, ..., 0.03175134, 0.03175134,
         0.03175134]]),
 array([[0.94895581, 0.94868949, 0.9483261 , ..., 0.56933792, 0.56345621,
         0.55757665],
        [0.94868949, 0.94852587, 0.94826218, ..., 0.57521209, 0.5693337 ,
         0.56345621],
        [0.9483261 , 0.94826218, 0.94810123, ..., 0.58108589, 0.57521209,
         0.56933792],
        ...,
        [0.56933792, 0.57521209, 0.58108589, ..., 0.94810123, 0.94826218,
         0.9483261 ],
        [0.5

In [ ]:
gp.